<a href="https://colab.research.google.com/github/patel-star/Gen-AI/blob/main/Classification_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
ResNet50_base = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
ResNet50_base.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
output = ResNet50_base.output
output = GlobalAveragePooling2D()(output)

fullyconnectedlayer_output = Dense(1024, activation='relu')(output)
prediction = Dense(2, activation='softmax')(fullyconnectedlayer_output)

model = Model(inputs=ResNet50_base.input, outputs=prediction)

for layer in ResNet50_base.layers:
  layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
train_path = '/content/drive/MyDrive/GenAI - 2025/DataSets/cat_dog_lite/training_set'
test_path = '/content/drive/MyDrive/GenAI - 2025/DataSets/cat_dog_lite/test_set'

image_dataset_from_directory(train_path, image_size=(64,64), shuffle=True)

Found 40 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
train_data_generators = ImageDataGenerator(zoom_range= 0.2, horizontal_flip= True, rescale= 1./255)
test_data_generators = ImageDataGenerator(rescale= 1./255)

train_set = train_data_generators.flow_from_directory(train_path, target_size= (64,64), batch_size= 32, class_mode= 'categorical')
test_set = test_data_generators.flow_from_directory(test_path, target_size= (64,64), batch_size= 32, class_mode= 'categorical')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
model.fit(train_set,
                    validation_data= test_set,
                    epochs= 10,
                    steps_per_epoch= len(train_set),
                    validation_steps= len(test_set))

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 9s/step - accuracy: 0.5417 - loss: 1.0373 - val_accuracy: 0.5000 - val_loss: 0.8190
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step - accuracy: 0.4250 - loss: 0.8601 - val_accuracy: 0.5000 - val_loss: 0.9950
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 371ms/step - accuracy: 0.5104 - loss: 0.9593 - val_accuracy: 0.5000 - val_loss: 0.8050
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 459ms/step - accuracy: 0.4958 - loss: 0.8308 - val_accuracy: 0.6000 - val_loss: 0.6837
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 381ms/step - accuracy: 0.5208 - loss: 0.7002 - val_accuracy: 0.6000 - val_loss: 0.6897
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.6833 - loss: 0.6485 - val_accuracy: 0.6000 - val_loss: 0.6709
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 590ms/step - accuracy: 0.5062 - loss: 0.7162 - val_accuracy: 0.6000 - val_loss: 0.6370
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 400ms/step - accuracy: 0.6250 - loss: 0.6539 - val_accuracy: 0.5000 - val_loss: 0.64